## Sample errors from made up Variogram model -> sample points -> create Variogram

In [4]:
%load_ext autoreload
%autoreload 2

from ocean_navigation_simulator.generative_error_model.models.OceanCurrentNoiseField import OceanCurrentNoiseField
from ocean_navigation_simulator.generative_error_model.Dataset import DatasetName, load_single_file
import numpy as np
import pandas as pd
import datetime

ranges = [1.0, 600.0, 600.0, 30.0] # -> go into simplex noise
rng = np.random.default_rng(12345678)
noise_field = OceanCurrentNoiseField()
noise_field.reset(rng)
t_delta_range = (datetime.timedelta(days=0), datetime.timedelta(days=1))
sample = noise_field.get_noise((-140, -120), (20, 30), t_delta_range, datetime.datetime(2022, 5, 22, 18, 30))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
display(sample)

<xarray.Dataset>
Dimensions:  (lon: 240, lat: 120, time: 24)
Coordinates:
  * lon      (lon) float64 -140.0 -139.9 -139.8 -139.7 ... -120.2 -120.2 -120.1
  * lat      (lat) float64 20.0 20.08 20.17 20.25 ... 29.67 29.75 29.83 29.92
  * time     (time) datetime64[ns] 2022-05-22T18:30:00 ... 2022-05-23T17:30:00
Data variables:
    u_error  (lon, lat, time) object 0.7327729738881905 ... -0.23694379879138364
    v_error  (lon, lat, time) object 0.07051247184211039 ... -2.0562244872583717
Attributes:
    description:  An ocean current error sample over time and space.

Sample ~30 points per time step. But these cannot be random but have to be a trajectory

In [6]:
dataset_name = DatasetName.AREA1
df = load_single_file(dataset_name)
df

Loaded: copernicus_forecast_error_lon_[-140, -120]_lat_[20.0, 30.0]_time_2022-05-22T18:30:00Z__2022-05-31T17:30:00Z.csv

Buoy Meta Data:
    Min time: 2022-05-22 19:00:00, max time: 2022-05-31 17:00:00
    Min lon: -139.988, max lon: -121.888
    Min lat: 20.0, max lat: 30.0
    Number or rows: 3967.



,time,lon,lat,u,v,buoy,file_name,u_forecast,v_forecast,u_error,v_error
0,2022-05-22 19:00:00,-123.426,22.111,0.063,-0.014,3201843,GL_TS_DC_3201843_202205,-0.106464,-0.147881,-0.169464,-0.133881
1,2022-05-22 20:00:00,-123.426,22.113,0.077,0.014,3201843,GL_TS_DC_3201843_202205,-0.111465,-0.152623,-0.188465,-0.166623
2,2022-05-22 21:00:00,-123.425,22.115,0.077,0.014,3201843,GL_TS_DC_3201843_202205,-0.117436,-0.154477,-0.194436,-0.168477
3,2022-05-22 22:00:00,-123.425,22.118,0.093,-0.014,3201843,GL_TS_DC_3201843_202205,-0.123334,-0.159261,-0.216334,-0.145261
4,2022-05-22 23:00:00,-123.426,22.120,0.062,-0.029,3201843,GL_TS_DC_3201843_202205,-0.130022,-0.166269,-0.192022,-0.137269
...,...,...,...,...,...,...,...,...,...,...,...
3962,2022-05-31 13:00:00,-135.923,23.049,0.022,0.070,5301550,GL_TS_DC_5301550,-0.020327,-0.088736,-0.042327,-0.158736
3963,2022-05-31 14:00:00,-135.919,23.049,0.023,0.091,5301550,GL_TS_DC_5301550,-0.016718,-0.082286,-0.039718,-0.173286
3964,2022-05-31 15:00:00,-135.917,23.051,0.025,0.113,5301550,GL_TS_DC_5301550,-0.013784,-0.073878,-0.038784,-0.186878
3965,2022-05-31 16:00:00,-135.912,23.052,0.016,0.142,5301550,GL_TS_DC_5301550,-0.016733,-0.067576,-0.032733,-0.209576


In [7]:
# interp the generated xarray to the positions of buoy points
from tqdm import tqdm

df["gen_u_error"] = 0
df["gen_v_error"] = 0
df["time"] = pd.to_datetime(df["time"])

n = 10
for i in tqdm(range(0, df.shape[0], n)):
    sample_interp = sample.interp(time=df.iloc[i:i+n]["time"],
                                  lon=df.iloc[i:i+n]["lon"],
                                  lat=df.iloc[i:i+n]["lat"])

    # add columns to df
    df["gen_u_error"].iloc[i:i+n] = sample_interp["u_error"].values.diagonal().diagonal()
    df["gen_v_error"].iloc[i:i+n] = sample_interp["v_error"].values.diagonal().diagonal()


  0%|          | 0/397 [00:00<?, ?it/s]/home/jonas/miniconda3/envs/ocean_platform/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
100%|██████████| 397/397 [02:54<00:00,  2.27it/s]


In [8]:
df

,time,lon,lat,u,v,buoy,file_name,u_forecast,v_forecast,u_error,v_error,gen_u_error,gen_v_error
0,2022-05-22 19:00:00,-123.426,22.111,0.063,-0.014,3201843,GL_TS_DC_3201843_202205,-0.106464,-0.147881,-0.169464,-0.133881,0.184986,1.731077
1,2022-05-22 20:00:00,-123.426,22.113,0.077,0.014,3201843,GL_TS_DC_3201843_202205,-0.111465,-0.152623,-0.188465,-0.166623,0.174388,1.733896
2,2022-05-22 21:00:00,-123.425,22.115,0.077,0.014,3201843,GL_TS_DC_3201843_202205,-0.117436,-0.154477,-0.194436,-0.168477,0.163833,1.736569
3,2022-05-22 22:00:00,-123.425,22.118,0.093,-0.014,3201843,GL_TS_DC_3201843_202205,-0.123334,-0.159261,-0.216334,-0.145261,0.153334,1.739093
4,2022-05-22 23:00:00,-123.426,22.120,0.062,-0.029,3201843,GL_TS_DC_3201843_202205,-0.130022,-0.166269,-0.192022,-0.137269,0.142904,1.741463
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3962,2022-05-31 13:00:00,-135.923,23.049,0.022,0.070,5301550,GL_TS_DC_5301550,-0.020327,-0.088736,-0.042327,-0.158736,-1.838946,-0.863829
3963,2022-05-31 14:00:00,-135.919,23.049,0.023,0.091,5301550,GL_TS_DC_5301550,-0.016718,-0.082286,-0.039718,-0.173286,-1.838946,-0.863829
3964,2022-05-31 15:00:00,-135.917,23.051,0.025,0.113,5301550,GL_TS_DC_5301550,-0.013784,-0.073878,-0.038784,-0.186878,-1.838946,-0.863829
3965,2022-05-31 16:00:00,-135.912,23.052,0.016,0.142,5301550,GL_TS_DC_5301550,-0.016733,-0.067576,-0.032733,-0.209576,-1.838946,-0.863829


## Change Variogram to pick columns for using different variables